### Setting up a Demo Foundry Agent via Code

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

### Setting up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

### Setting up the Foundry Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Setting up our Batman Agent

In [ ]:
from azure.ai.projects.models import PromptAgentDefinition

agent_name = "batman-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are Batman, the Dark Knight of Gotham City. Respond to all queries in character as Batman would.",
    ),
)

# printing the agent id
print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

### Creating a Conversation Object for Agent Chat System

In [ ]:
# creating an openai client first
openai_client = project_client.get_openai_client()

# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

### Chat with the Agent

In [ ]:
chat = True

while chat:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        chat = False
        print("Exiting chat. Goodbye!")
    else:
        response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body = {
                "agent": {
                    "name": agent_name,
                    "type": "agent_reference"
                }
            },
            input = user_input
        )

        print(f"Batman: {response.output_text}")